### cine21 사이트에서 배우 랭킹 정보 크롤링하기

- http://www.cine21.com/rank/person 접속
- 개발자 도구 실행(F12) 
- NetWork 탭 선택
- Preserve log 선택(체크) 확인
- All 선택 확인
- 설정이 되어있지 않으면 위와 같이 설정하고
- 개발자 도구를 닫은 후 다시 연다
- cine21 화면 중에 
- 집계기간 1개월| 6개월| 1년| 2년 <- 에서 6개월 선택
- Name 밑에 나오는 목록 중에서 content 선택
- 오른쪽 메뉴 중 Headers 선택
- General 의 하위 목록 확인
- Request URL: http://www.cine21.com/rank/person/content
- Request Method: POST
- Status Code: 200 OK
- Remote Address: 115.68.232.4:80
- Referrer Policy: strict-origin-when-cross-origin
- 집계기간 1개월| 6개월| 1년| 2년 <- 에서 2년 선택
- Name 밑에 나오는 목록 중에서 content 선택
- 오른쪽 메뉴 중 Headers 선택
- Request URL: http://www.cine21.com/rank/person/content
- Request Method: POST
- Status Code: 200 OK
- Remote Address: 115.68.232.4:80
- Referrer Policy: strict-origin-when-cross-origin
- 오른쪽 메뉴 중 Payload 선택 및 확인
- section: actor
- period_start: 2020-09
- gender: all
- page: 1

<br>

- 사이트 주소 (URL) : http://www.cine21.com/rank/person
- Request URL: http://www.cine21.com/rank/person/content

- section: actor
- period_start: 2020-09
- gender: all
- page: 1

In [ ]:
import pymongo
from bs4 import BeautifulSoup
import requests
import re

In [ ]:
'''
1) MongoDB 연결하기
2) Database 생성하기
3) Collection 생성하기
'''
conn = pymongo.MongoClient()
cine = conn.cine
collection_cine = cine.collection_cine

In [ ]:
'''
crawling 할 URL 요청하기
'''
url = "http://www.cine21.com/rank/person/content"

'''
비어있는 dictionary 생성하기
section: actor
period_start: 2020-09
gender: all
page: 1
'''
post_data = {}
post_data["section"]="actor"
post_data["period_start"]="2020-09"
post_data["gender"]="all"
post_data["page"]=1
print(post_data)

In [ ]:
'''
post 방식으로 request 보내기
'''
resp = requests.post(url, post_data)
print(resp)

In [ ]:
resp.content

### 응답(Response) 받은 문서를 parsing 해서 원하는 data 추출하기

In [ ]:
soup = BeautifulSoup(resp.content, "html.parser")

In [ ]:
soup

In [ ]:
actors = soup.select("li.people_li div.name")

for actor in actors:
    print(actor)

In [ ]:
'''
actor.text : content 만 추출함
'''
for actor in actors:
    print(actor.text)

In [ ]:
print(type(actor.text))

In [ ]:
'''
배우이름만 추출함 : 문자열의 split() 메소드 호출
'''
for actor in actors:
    print((actor.text).split("("))

In [ ]:
for actor in actors:
    print((actor.text).split("(")[0])

In [ ]:
for actor in actors:
    print(actor.select_one("a"))

In [ ]:
'''
http://www.cine21.com/db/person/info/?person_id=15222
'''

In [ ]:
for actor in actors:
    print(actor.select_one("a").attrs["href"])

In [ ]:
for actor in actors:
    print("http://www.cine21.com" + actor.select_one("a").attrs["href"])

### 정규표현식 : Regualr Expression
```html

  https://regexr.com/

  <li><span class="tit">직업</span>배우</li>

  - .  : \n(줄바꿈)문자를 제외한 모든 문자 하나를 의미함
  - *  : 앞 문자가 반복되는 pattern
  - .* : 문자가 반복되는 pattern


```

In [ ]:
'''
get 방식으로 request 보내기
  ㄴ requests 모듈에 있는 get() 함수를 호출함
  
  ul.default_info
    <ul class="default_info">
        <li><span class="tit">직업</span>배우</li>
        <li><span class="tit">생년월일</span>1977-01-26</li>

        <li><span class="tit">성별</span>남</li>


        <li><span class="tit">신장/체중</span>177cm, 65kg</li>
        <li><span class="tit">학교</span>남서울대학교 영어학 (중퇴)</li>
        <li><span class="tit">취미</span>영화감상, 음악감상, 컴퓨터게임</li>
        <li><span class="tit">특기</span>노래, 기타연주, 탁구, 야구</li>

        <li><span class="tit">소속사</span>휴메인엔터테인먼트</li>
    </ul>
    
'''

# 비어있는 list 생성
actors_info_list = []

for actor in actors:
    actor_link = "http://www.cine21.com" + actor.select_one("a").attrs["href"]
    resp_actor = requests.get(actor_link)
    soup_actor = BeautifulSoup(resp_actor.content, "html.parser")
    
    default_info = soup_actor.select_one("ul.default_info")
    details_actor = default_info.select("li")
    
    # 비어있는 dictionary 생성
    actors_info_dict = {}
    
    for actor_item in details_actor:
        actor_item_key = actor_item.select_one("span.tit").text
        actor_item_value = re.sub("<span.*?>.*?</span>","",str(actor_item))
        actor_item_value = re.sub("<.*?>","",actor_item_value)
        actors_info_dict[actor_item_key] = actor_item_value
    
    actors_info_list.append(actors_info_dict)
    
actors_info_list
    

### 각 배우들의 흥행지수 / 출연 영화

In [ ]:
actors = soup.select("li.people_li div.name")
hits = soup.select("ul.num_info > li > strong")
movies = soup.select("ul.mov_list")

for actor in actors:
    print(actor)
    
for hit in hits:
    print(hit)
    
print('hits[0] :',hits[0]) 
print('-' * 60)


In [ ]:

for idx, actor in enumerate(actors):
    print("배우이름 :",(actor.text).split("(")[0])
    print("흥행지수 :",int(hits[idx].text.replace(",","")))
    movie_titles = movies[idx].select("li a span")
    
    movie_titles_list = []    
    for movie_title in movie_titles:
        movie_titles_list.append(movie_title.text)
        
    print("출연영화 :",movie_titles_list)
    print("-" * 80)

### 수집한 data 로 dictionary 생성하기
-  흥행지수 + 출연영화 + 배우상세정보

In [ ]:
actors_info_list = []

actors = soup.select("li.people_li div.name")
hits = soup.select("ul.num_info > li > strong")
movies = soup.select("ul.mov_list")

for idx, actor in enumerate(actors):
    actor_name = (actor.text).split("(")[0]
    actor_hits  = int(hits[idx].text.replace(",", ""))
    movie_titles = movies[idx].select("li a span")
    
    movie_title_list = []
    for movie_title in movie_titles:
        movie_title_list.append(movie_title.text)
        
    actors_info_dict = {}      
    actors_info_dict["배우이름"] = actor_name 
    actors_info_dict["흥행지수"] = actor_hits
    actors_info_dict["출연영화"] = movie_title_list
    
    actor_link = "http://www.cine21.com" + actor.select_one("a").attrs["href"]
    actor_resp = requests.get(actor_link)
    actor_soup = BeautifulSoup(actor_resp.content, "html.parser")    
    actor_details = default_info.select("li")
    
    for actor_item in actor_details:
        actor_item_key = actor_item.select_one("span.tit").text
        actor_item_value = re.sub("<sapn.*?>.*?</span>", "", str(actor_item))
        actor_item_value = re.sub("<.*?>", "", actor_item_value)
        
        actors_info_dict[actor_item_key] = actor_item_value
        
    actors_info_list.append(actors_info_dict)    
    
actors_info_list    